In [82]:
import pandas as pd

In [83]:
enrolment_df = pd.read_csv("../data/processed/interim/enrolment_raw_merged.csv")


In [84]:
enrolment_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-03-2025,Meghalaya,East Khasi Hills,793121,11,61,37
1,09-03-2025,Karnataka,Bengaluru Urban,560043,14,33,39
2,09-03-2025,Uttar Pradesh,Kanpur Nagar,208001,29,82,12
3,09-03-2025,Uttar Pradesh,Aligarh,202133,62,29,15
4,09-03-2025,Karnataka,Bengaluru Urban,560016,14,16,21


In [85]:
enrolment_df.columns

Index(['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17',
       'age_18_greater'],
      dtype='object')

In [86]:
enrolment_df = enrolment_df.rename(columns={"age_18_greater": "age_17_plus"})

In [87]:
enrolment_df["state"] = enrolment_df["state"].str.strip().str.title()
enrolment_df["district"] = enrolment_df["district"].str.strip().str.title()


In [88]:
enrolment_df.isnull().sum()

date           0
state          0
district       0
pincode        0
age_0_5        0
age_5_17       0
age_17_plus    0
dtype: int64

In [89]:
enrolment_df.shape


(1006029, 7)

In [90]:
enrolment_df = enrolment_df.drop_duplicates()

In [91]:
enrolment_df["date"] = pd.to_datetime(enrolment_df["date"], dayfirst=True)


In [92]:
enrolment_df.dtypes

date           datetime64[ns]
state                  object
district               object
pincode                 int64
age_0_5                 int64
age_5_17                int64
age_17_plus             int64
dtype: object

In [93]:
enrolment_df.shape


(983000, 7)

In [94]:
enrolment_df[enrolment_df["district"] == "100000"]


,date,state,district,pincode,age_0_5,age_5_17,age_17_plus
23108,2025-09-02,100000,100000,100000,0,0,3
46946,2025-09-03,100000,100000,100000,0,0,1
97816,2025-09-08,100000,100000,100000,0,0,1
115798,2025-09-09,100000,100000,100000,0,0,1
153156,2025-09-11,100000,100000,100000,0,0,2
160195,2025-09-12,100000,100000,100000,0,0,2
261778,2025-09-19,100000,100000,100000,0,0,1
272731,2025-09-20,100000,100000,100000,0,0,1
470934,2025-10-24,100000,100000,100000,0,1,0
762744,2025-11-15,100000,100000,100000,0,0,3


In [95]:
enrolment_df[enrolment_df["district"] == "100000"].shape[0]

21

In [96]:
enrolment_df[enrolment_df["state"] == "100000"]

,date,state,district,pincode,age_0_5,age_5_17,age_17_plus
23108,2025-09-02,100000,100000,100000,0,0,3
46946,2025-09-03,100000,100000,100000,0,0,1
97816,2025-09-08,100000,100000,100000,0,0,1
115798,2025-09-09,100000,100000,100000,0,0,1
153156,2025-09-11,100000,100000,100000,0,0,2
160195,2025-09-12,100000,100000,100000,0,0,2
261778,2025-09-19,100000,100000,100000,0,0,1
272731,2025-09-20,100000,100000,100000,0,0,1
470934,2025-10-24,100000,100000,100000,0,1,0
762744,2025-11-15,100000,100000,100000,0,0,3


In [97]:
enrolment_df[enrolment_df["state"] == "100000"].shape[0]

21

In [98]:
enrolment_df = enrolment_df[
    ~(
        enrolment_df["district"].isin(["100000", 100000]) |
        enrolment_df["state"].isin(["100000", 100000])
    )
]


In [99]:
enrolment_df.shape


(982979, 7)

In [100]:
enrolment_df["state"].value_counts()

state
Uttar Pradesh                                   108066
Tamil Nadu                                       90388
Maharashtra                                      75531
West Bengal                                      75336
Karnataka                                        68756
Andhra Pradesh                                   63836
Bihar                                            58542
Rajasthan                                        54864
Madhya Pradesh                                   49562
Gujarat                                          45243
Odisha                                           42695
Telangana                                        41878
Kerala                                           38581
Assam                                            30699
Jharkhand                                        22651
Punjab                                           20081
Chhattisgarh                                     17981
Haryana                                          15522
Jamm

In [101]:
enrolment_state_mapping = {
    "Orissa": "Odisha",
    "Jammu & Kashmir": "Jammu And Kashmir",
    "Pondicherry": "Puducherry",

    "West  Bengal": "West Bengal",
    "West Bangal": "West Bengal",
    "Westbengal": "West Bengal",

    "Dadra And Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Dadra & Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman And Diu": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman & Diu": "Dadra And Nagar Haveli And Daman And Diu",
    "The Dadra And Nagar Haveli And Daman And Diu": "Dadra And Nagar Haveli And Daman And Diu",

    "Andaman & Nicobar Islands": "Andaman And Nicobar Islands"
}


In [102]:
# -----------------------------
# Fix Hyderabad → Telangana
# -----------------------------

mask = (
    enrolment_df["district"].astype(str).str.strip().str.title() == "Hyderabad"
)

affected_rows = mask.sum()

enrolment_df.loc[mask, "state"] = "Telangana"

print(f"✔ Hyderabad correction applied to {affected_rows} rows")


✔ Hyderabad correction applied to 4866 rows


In [103]:
# -----------------------------
# Fix Adilabad → Telangana
# -----------------------------

mask = (
    enrolment_df["district"].astype(str).str.strip().str.title() == "Adilabad"
)

affected_rows = mask.sum()

enrolment_df.loc[mask, "state"] = "Telangana"

print(f"✔ Adilabad correction applied to {affected_rows} rows")


✔ Adilabad correction applied to 2093 rows


In [104]:
enrolment_df["state"] = enrolment_df["state"].replace(enrolment_state_mapping)


In [105]:
enrolment_df["state"].value_counts()

state
Uttar Pradesh                               108066
Tamil Nadu                                   90388
Maharashtra                                  75531
West Bengal                                  75367
Karnataka                                    68756
Andhra Pradesh                               61149
Bihar                                        58542
Rajasthan                                    54864
Madhya Pradesh                               49562
Odisha                                       45935
Gujarat                                      45243
Telangana                                    44565
Kerala                                       38581
Assam                                        30699
Jharkhand                                    22651
Punjab                                       20081
Chhattisgarh                                 17981
Haryana                                      15522
Jammu And Kashmir                            11194
Himachal Pradesh         

In [106]:
enrolment_df.to_csv(
    "../data/processed/cleaned/enrolment_clean.csv",
    index=False
)


In [107]:
# import pandas as pd
# import nbformat
# from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell
# from pathlib import Path
#
# # =====================================================
# # Paths
# # =====================================================
# CLEAN_DIR = Path("../data/processed/cleaned")
# NOTEBOOK_DIR = Path("../Notebooks/state_wise_cleaning")
# REFERENCE_DIR = Path("../data/processed/reference")
#
# NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
# REFERENCE_DIR.mkdir(parents=True, exist_ok=True)
#
# # =====================================================
# # Load enrolment data to get states (source of truth)
# # =====================================================
# enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
#
# enrol_df["state"] = enrol_df["state"].astype(str).str.strip().str.title()
# enrol_df["district"] = enrol_df["district"].astype(str).str.strip().str.title()
#
# states = sorted(enrol_df["state"].dropna().unique())
#
# print(f"Creating notebooks for {len(states)} states")
#
#
# # =====================================================
# # Notebook template generation
# # =====================================================
# for state in states:
#     safe_state = state.replace(" ", "_")
#     nb_path = NOTEBOOK_DIR / f"{safe_state}_district_cleaning.ipynb"
#
#     nb = new_notebook(cells=[
#
#         # ----------------------------
#         # Markdown intro
#         # ----------------------------
#         new_markdown_cell(
#             f"# District Cleaning — {state}\n\n"
#             "This notebook standardizes **district names** for this state across:\n"
#             "- Enrolment data\n"
#             "- Demographic update data\n"
#             "- Biometric update data\n\n"
#             "**All data is saved back to the same cleaned files.**"
#         ),
#
#         # ----------------------------
#         # Load datasets
#         # ----------------------------
#         new_code_cell(
#             """import pandas as pd
# from pathlib import Path
#
# pd.set_option("display.max_rows", None)
# pd.set_option("display.width", None)
#
# CLEAN_DIR = Path("../../data/processed/cleaned")
#
# enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
# demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
# bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")
#
# for df in [enrol_df, demo_df, bio_df]:
#     df["state"] = df["state"].astype(str).str.strip().str.title()
#     df["district"] = df["district"].astype(str).str.strip().str.title()
#
# print("✅ All datasets loaded and normalized (Title Case)")
# """
#         ),
#
#         # ----------------------------
#         # Print districts (FULL HEIGHT)
#         # ----------------------------
#         new_code_cell(
#             f"""STATE_NAME = "{state}"
#
# districts = sorted(
#     set(
#         enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
#         .tolist()
#     )
# )
#
# print(f"State: {{STATE_NAME}}")
# print(f"Number of unique districts: {{len(districts)}}")
#
# pd.DataFrame(
#     {{"District Name": districts}}
# )
# """
#         ),
#
#         # ----------------------------
#         # Mapping instructions
#         # ----------------------------
#         new_markdown_cell(
#             "## District Mapping\n\n"
#             "Add mappings in **Title Case only**.\n\n"
#             "Format:\n"
#             "```python\n"
#             "DISTRICT_MAPPING = {\n"
#             "    \"Correct District\": [\"Wrong Name 1\", \"Wrong Name 2\"],\n"
#             "}\n"
#             "```"
#         ),
#
#         # ----------------------------
#         # Apply mapping (SAFE for all datasets)
#         # ----------------------------
#         new_code_cell(
#             f"""DISTRICT_MAPPING = {{
#     # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
# }}
#
# def apply_mapping(df, state, mapping, label):
#     total = 0
#     for correct, wrongs in mapping.items():
#         mask = (
#             (df["state"] == state) &
#             (df["district"].isin(wrongs))
#         )
#         count = mask.sum()
#         df.loc[mask, "district"] = correct
#         total += count
#         if count > 0:
#             print(f"✔ {{label}} → {{correct}} : {{count}} rows fixed")
#     return total
#
# total_fixes = 0
# total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
# total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
# total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")
#
# print(f"✅ Total fixes in {{STATE_NAME}}: {{total_fixes}}")
# """
#         ),
#
#         # ----------------------------
#         # Save back to SAME files
#         # ----------------------------
#         new_code_cell(
#             """enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
# demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
# bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)
#
# print("💾 All cleaned files saved successfully (overwritten)")
# """
#         )
#     ])
#
#     with open(nb_path, "w", encoding="utf-8") as f:
#         nbformat.write(nb, f)
#
#     print(f"✔ Created: {nb_path}")
#
# print("🎉 All state-wise district cleaning notebooks generated successfully.")
